# Report on Adversarial Examples
This notebook should 
1. Train new models with different parameters
1. for each model, generate both a **targeted** and **untargeted** PGD attack
1. Plot the results

# Problem: data pre-processing messing with which=2, 4

In [ ]:
%load_ext autoreload
%autoreload 2

import subprocess
from pathlib import Path 
from matplotlib import pyplot as plt 
import json 
import optuna 
from model_utils import find_best_model, choose_model

prefix = 'three_classes_2/'
adv_dir = 'attacks/'

## Utilities for reporting accuracies

In [ ]:
def load_stats(dir):
    with open(dir) as f:
        stats = json.load(f)
    return stats 

def get_stats_regular(which=1, duration=100, model='max', root=prefix, dir=None):
    """ 
    Load stats of model specified directly using the directory of the model, or using which, duration and model name ('max' for latest)
    """
    if dir is None:
        dir = Path(root) / f'{which=}_{duration=}'
    model = choose_model(dir, model)
    dir = Path(model)
    return load_stats(dir / 'stats.json') 

def get_stats_attack(which=1, duration=100, model='max', root=prefix, adv_dir=adv_dir, dir=None):
    """ 
    Load adversarial stats of model specified directly using the directory of the model, 
    or using which, duration and model name ('max' for latest).
    Returns a dict of dicts, where the outer dict has attack types as keys and the inner uses perturbation sizes
    """
    if dir is None:
        # dir is not specified so we find it based on other information
        dir = Path(root) / f'{which=}_{duration=}'
        model = choose_model(dir, model)
        dir = model / adv_dir 
    # create a dict of dicts based on attack type and perturbation size
    retval = {}
    for p in dir.iterdir():
        key = str(p).split('/')[-1]
        perturbation_size = key.split('=')[-1]
        if "attack_type='targeted'" in key:
            if not 'targeted' in retval.keys():
                retval['targeted'] = {}
            retval['targeted'][perturbation_size] = load_stats(p / 'adv_stats.json')
        elif "attack_type='untargeted'" in key:
            if not 'untargeted' in retval.keys():
                retval['untargeted'] = {}
            retval['untargeted'][perturbation_size] = load_stats(p / 'adv_stats.json')
        elif "attack_type='targeted_increase'" in key:
            if not 'targeted_increase' in retval.keys():
                retval['targeted_increase'] = {}
            retval['targeted_increase'][perturbation_size] = load_stats(p / 'adv_stats.json')
        elif "attack_type='targeted_decrease'" in key:
            if not 'targeted_decrease' in retval.keys():
                retval['targeted_decrease'] = {}
            retval['targeted_decrease'][perturbation_size] = load_stats(p / 'adv_stats.json')
        else:
            raise NotImplementedError(f'stats not implemented for key {key}')
    return retval

## Train New Models
Each with a specific set of parameters, including data parameters

In [ ]:
def make_args(param):
    """
    turn parameter dictionary into list of arguments
    """
    out = []
    for k, v in param.items():
        out.append('--' + k)
        out.append(str(v))
    return out 

durations = [50, 100, 150]
whichs = [1, 3]
n_classes = 3
n_epochs = 30
hs = 64 
dropout = 1/3
num_layers = 2
opt_time_seconds = 15*60
model_params = {
    f'{prefix}{which=}_{duration=}': {
        'duration': duration, 'which': which, 'n_classes': n_classes, 'n_epochs': n_epochs,
        'dropout': dropout, 'hs': hs, 'num_layers': num_layers, 
    } 
    for duration in durations for which in whichs
}



In [ ]:
# create models
for name, v in model_params.items():
    which = v['which']
    duration = v['duration']
    def objective(trial):
        v['dropout'] = trial.suggest_float('dropout', 0, .5)
        v['num_layers'] = trial.suggest_int('num_layers', 1, 4)
        v['hs'] = trial.suggest_int('hs', 64, 256, log=True)
        print('\n\n', v, '\n\n')
        subprocess.run(['python', 'lstm.py'] + make_args(v) + ['--target_directory', name]) 
        acc = get_stats_regular(which=which, duration=duration)['acc']
        return -acc # maximize acc

    study = optuna.create_study()
    study.optimize(objective, timeout=opt_time_seconds)

In [ ]:
# find the best accuracy in the directories
best_accs = {}
for name, v in model_params.items():
    best_accs[name] = find_best_model(name)
print(*[v[1] for v in best_accs.values()], sep='\n')


## Generate Adversarial Examples
Save the results with the model in the appropriate folder, both for targeted and untargeted attacks. 

Parameters of the attack:
- Attack type (targeted or untargeted)
- Perturbation size

### Issues:
- There is a bug somewhere because the perturbation_sizes are not respected according to the adv_stats.json
- We should implement a keyword for fooling models towards saying models are in better or worse conditions than they really are

In [ ]:
# For the case of breaking the process, it is nice to know the order of the scripts executed
# Generate that order
order = []
for name in model_params.keys():
    attack_types = ['targeted', 'untargeted']
    perturbation_sizes = [.05, .1, .5]# [.01, .05, .1]

    attack_params = {
        f'{adv_dir}{attack_type=}_{perturbation_size=}/': {'attack_type': attack_type, 'perturbation_size': perturbation_size} 
        for attack_type in attack_types for perturbation_size in perturbation_sizes
    }
    order.extend([(name, attack_name) for attack_name in attack_params.keys()])

def later_experiment(comp, ref, order=order):
    """
    Check if comp is later than ref in order
    """
    if ref == None:
        return True
    j = len(order)
    for i, o in enumerate(order):
        if str(o[0]) == str(comp[0]) and str(o[1]) == str(comp[1]):
            j = i 
            break 
    for o in order[:j+1]:
        if str(o[0]) == str(ref[0]) and str(o[1]) == str(ref[1]):
            return True 
    return False 

In [ ]:
# for each model, make a number of attacks 
# To that end, make a subfolder "attacks" for each of the models

# ref = (f'{prefix}/which=4_duration=150', "attacks/attack_type='targeted'_perturbation_size=0.01/") 
ref = None

for name in model_params.keys():
    attack_types = ['targeted_increase', 'targeted_decrease', 'targeted', 'untargeted']
    perturbation_sizes = [.1*(i+1) for i in range(5)] 

    attack_params = {
        f'{adv_dir}{attack_type=}_{perturbation_size=}/': {'attack_type': attack_type, 'perturbation_size': perturbation_size} 
        for attack_type in attack_types for perturbation_size in perturbation_sizes
    }
    for attack_name, v in attack_params.items():
        if later_experiment((name, attack_name), ref):
            print('doing', (name, attack_name))
            if 'increase' in name:
                preference = 'increase'
            elif 'decrease' in name:
                preference = 'decrease'
            else:
                preference = 'None'
            subprocess.run(['python', 'attack.py'] + make_args(v) + ['--source_directory', name, '--target_directory', attack_name, '--model_name', 'best', '--preference', preference])
        else:
            print('skipped', (name, attack_name))

## Plot the Results

In [ ]:
# get accs
regular_accs = {}
adv_accs = {}
for which in whichs:
    for duration in durations:
        key = f'{which=}_{duration=}'
        regular_accs[key] = get_stats_regular(which, duration, model='best')
        adv_accs[key] = get_stats_attack(which, duration, model='best')

# plot targeted results, resulting in a plot per "which"
# x=perturbation_size, y=acc
# where we have one line for each duration
x = [0.] + perturbation_sizes 
for which in whichs:
    fig, ax = plt.subplots(1, 1)
    ax.set_title(f'targeted_{which=}')
    for duration in durations:
        key = f'{which=}_{duration=}'
        y = [regular_accs[key]['acc']] + [adv_accs[key]['targeted'][str(s)]['acc'] for s in perturbation_sizes]
        ax.plot(x, y, label=key)
    ax.legend()
    plt.show()


    # Do the same for the untargeted results
    fig, ax = plt.subplots(1, 1)
    ax.set_title(f'untargeted_{which=}')
    for duration in durations:
        key = f'{which=}_{duration=}'
        y = [regular_accs[key]['acc']] + [adv_accs[key]['untargeted'][str(s)]['acc'] for s in perturbation_sizes]
        ax.plot(x, y, label=key)
    ax.legend()
    plt.show()

    # Do the same for the untargeted results
    fig, ax = plt.subplots(1, 1)
    ax.set_title(f'incr_{which=}')
    for duration in durations:
        key = f'{which=}_{duration=}'
        y = [regular_accs[key]['acc']] + [adv_accs[key]['targeted_increase'][str(s)]['acc'] for s in perturbation_sizes]
        ax.plot(x, y, label=key)
    ax.legend()
    plt.show()

    # Do the same for the untargeted results
    fig, ax = plt.subplots(1, 1)
    ax.set_title(f'decr_{which=}')
    for duration in durations:
        key = f'{which=}_{duration=}'
        y = [regular_accs[key]['acc']] + [adv_accs[key]['targeted_decrease'][str(s)]['acc'] for s in perturbation_sizes]
        ax.plot(x, y, label=key)
    ax.legend()
    plt.show()